In [ ]:
import pandas as pd
import numpy as np
import random
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve
from sklearn.decomposition import PCA
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
import tensorflow as tf
import seaborn as sns
from sklearn.ensemble import IsolationForest
os.chdir('Resources/')

In [ ]:
df = pd.read_csv('6_Preprocessed_Data.csv')

X = df.drop(['HeartDisease'], axis='columns')
Y = df[['HeartDisease']]

In [ ]:
seed = 1341

mx=8.154457525222528
ind=232

for i in range(768, 5000):
    seed = i
    
    full = []
    
    for j in range(1, 10):
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1*j, random_state=seed)
        
        clf = IsolationForest(n_estimators=26, contamination=0.01 * 1, random_state=seed, max_samples=105)
        outliers = clf.fit_predict(X_train)
        X_train = X_train[outliers == 1]
        Y_train = Y_train[outliers == 1]

        corr_matrix = X_train.corr().abs()
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
        X_train_selected = X_train.drop(to_drop, axis=1)
        X_test_selected = X_test.drop(to_drop, axis=1)

        rf_classifier = RandomForestClassifier(n_estimators=63, random_state=seed, criterion='gini')
        rf_classifier.fit(X_train_selected, Y_train.values.ravel())

        Y_pred = rf_classifier.predict(X_test_selected)

        accuracy_isolation = f1_score(Y_test, Y_pred)

        full.append(accuracy_isolation)

    accuracy_nb = sum(full)

    if(accuracy_nb>mx):
        mx=accuracy_nb
        ind=i
    
    print(i, accuracy_nb, mx, ind)

759 8.02861888366454 8.154457525222528 232
760 7.986109851746364 8.154457525222528 232
761 8.04397119462685 8.154457525222528 232
762 8.08257224318784 8.154457525222528 232
763 8.039373118310762 8.154457525222528 232
764 8.007755415449415 8.154457525222528 232
765 8.05592143335107 8.154457525222528 232
766 8.042479803268648 8.154457525222528 232
767 7.979839854575708 8.154457525222528 232


In [ ]:
print(mx, ind)